In [ ]:
%matplotlib notebook

import os
import datetime as dt
import pickle, joblib


# Standard data science libraries
import pandas as pd
import numpy as np
import scipy.stats as ss
import scipy.optimize as so 
import scipy.interpolate as si

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-notebook')

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 50

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.display import Image
from IPython.display import Math



In [ ]:
import flopy as fp

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
fig_dir = './'
model_dir = '../models'

mfpth = 'C:/WRDAPP/MODFLOW-NWT_1.1.2/bin/MODFLOW-NWT_64.exe'
mp_exe_name = 'C:/WRDAPP/modpath.6_0/bin/mp6.exe' 

In [ ]:
df = pd.DataFrame()

nwt = True

for dirpath, dirnames, filenames in os.walk(model_dir):
    model = dirpath[len(model_dir) + 1:]
    for f in filenames:
        if f.endswith('lst'):

            pth = os.path.join(dirpath, f)
            with open(pth, 'r') as src:
                lf = src.readlines()

            last_line = [item for item in lf if 'Elapsed run time' in item]
            if len(last_line) != 0:
                df.loc[model, 'Elapsed time'] = last_line[0].strip().split(':')[
                    1].strip()
                
            perc_disc = [item for item in lf if 'PERCENT DISCREPANCY =' in item]
            if len(perc_disc) != 0:
                df.loc[model, 'Mass error'] = np.float32(perc_disc[0].split()[-1])
                
            run_date = [item for item in lf if 'Run end date and time' in item]
            if len(run_date) != 0:
                df.loc[model, 'Run date'] = run_date[0][46:].strip()

            df.loc[model, 'Failed'] = any([('FAILED' in item) for item in lf])

#             if nwt:
#                 ml = fp.mf6.MFSimulation.load(sim_name=sim_name, version='mf6', exe_name=mfpth6,
#                                       sim_ws=model_ws, strict=True, verbosity_level=0, load_only=None, verify_data=False)

#                 model = ml.get_model()
#             print(last_line, perc_disc)
                
#                 df.loc[model, 'Head tol'] = ml.NWT.headtol
#                 df.loc[model, 'Flux tol'] = ml.NWT.fluxtol
#                 df.loc[model, 'Max iter'] = ml.NWT.maxiterout
#     except:
#         print('{} may not have a valid completed run (terminated prematurely for example)'.format(model))

df['Elapsed time'] = pd.to_timedelta(df['Elapsed time'])
df['Elapsed time'] = (df['Elapsed time'] + pd.to_datetime('01/01/2000', format="%d/%m/%Y")).dt.strftime('%H:%M:%S')
df['Run date'] = pd.to_datetime(df['Run date'])

dst = os.path.join(fig_dir, 'model_time_and_error.csv')
df.to_csv(dst)
df